In [ ]:
import random
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

from functions import *

In [ ]:
# задаем число маяков
nanchors = 4

# число углов комнаты
ncorners = 4

# шаг сетки
stepx = 0.1
stepy = 0.1

# массив координат по х и у
xcoord = np.array((0.5, 6.5, 3.5, 1.5))
ycoord = np.array((1.5, 2.5, 6.5, 3.5))

# xcoord = np.array((0.5, 3.5, 3.5, 0.5))
# ycoord = np.array((0.5, 0.5, 3.5, 3.5))

In [ ]:
# формируем комнату
room = createRoom(xcoord, ycoord)
# формируем точки внутри помещения
pointsIn, pointsOut = createPoints(stepx, stepy, xcoord, ycoord)

# нормировка комнаты
roomScaler, pointsInScaler, pointsOutScaler, scaler = standardScalerPoints(xcoord, ycoord, pointsIn, pointsOut)
# нормировка по координатам
xcoordScaler = roomScaler[:, 0]
ycoordScaler = roomScaler[:, 1]

In [ ]:
# генерируем координаты маяков
# можно не сравнивать со всеми точками внутри - т.к. места маяков не попадают на сетку
coordAnchorsScaler = createCoordAnchors(nanchors, xcoordScaler, ycoordScaler)

# подумать нужны ли те точки, которые мы хотим видеть pointsVisible
DOP = estimateDOP(roomScaler, pointsInScaler, coordAnchorsScaler, nanchors, DOPmax=20)

In [ ]:
plotRoom(roomScaler, pointsInScaler, pointsOutScaler)
plotAnchors(coordAnchorsScaler, roomScaler)
plotDOPfactor(roomScaler, coordAnchorsScaler, DOP, pointsInScaler)

In [ ]:
polyCoordAnchorsScaler = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, coordAnchorsScaler)

In [ ]:
DOP = estimateDOP(roomScaler, pointsInScaler, polyCoordAnchorsScaler, nanchors, DOPmax=20)

In [ ]:
plotAnchors(polyCoordAnchorsScaler, roomScaler)
plotDOPfactor(roomScaler, polyCoordAnchorsScaler, DOP, pointsInScaler)

In [ ]:
npopul = 100     # Размер популяции
nsurv = 20       # Количество выживших (столько лучших переходит в новую популяцию)
nnew = npopul - nsurv # Количество новых (столько новых ботов создается)
l = nanchors * 2         # Длина бота
epohs = 300   # Количество эпох

mut = 0.3       # Коэфициент мутаций

popul = []       # Двумерный массив популяции, размерностью [n, l]. 100 ботов по 8 компонентов каждый
val = []         # Одномерный массив значений этих ботов

# Проходим по всей длине популяции
for i in range(npopul):

    bot = createCoordAnchors(nanchors, xcoordScaler, ycoordScaler)
    bot = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, bot)
    popul.append(bot)

    # Пробегаемся по всем эпохам
for it in range(epohs):

    # Создаем пустой список для значений ботов
    val = []

    # Проходим по всей длине популяции
    for bot in popul:
        # Берем очередного бота
        DOP = estimateDOP(roomScaler, pointsInScaler, bot, nanchors, DOPmax=20)
        val.append(sum(DOP) / len(DOP))

        # Получаем новую популяцию и сортированный список значнией
    newpopul, sval = getSurvPopul(popul, val, nsurv, 0)
    # Выводим 5 лучших ботов
    print(it, " ", [round(s,8) for s in sval[0:5]])
    if it % 5 == 0:
        plotAnchors(newpopul[0], roomScaler)

    # Проходимся в цикле nnew-раз
    for i in range(nnew):

        # Из newpopul(новой популяции) получаем двух случайных родителей-ботов
        botp1, botp2 = getParents(newpopul, nsurv)
        # Массив для нового бота
        newbot = []

        # проходимся по длине бота и осуществляем смешивание/скрещивание от родителей
        for j in range(l):

            # Получаем значение для j-ого компонента бота
            x = crossPointFrom2Parents(botp1.reshape(l), botp2.reshape(l), j)
            # Добавляем к значению бота случайную величину, зависящую от коэфециента мутации
            x += mut*(2*random.random() - 1.0)
            # Добавляем новое значение в бота
            newbot.append(x) # Добавялем новое значение в бота

        # Добавляем бота в новую популяцию
        newbot = np.array(newbot).reshape((nanchors,2))
        newbot = polyCoordAnchors(xcoordScaler, ycoordScaler, roomScaler, newbot)
        newpopul.append(newbot)

        #(таким образом к nsurv-лучших ботов предыдующей популяции добавится nnew-новых ботов)

    # Записываем в popul посчитанную новую популяцию
    popul = newpopul